<a href="https://colab.research.google.com/github/yasirfaizahmed/Text-Classification-models/blob/main/Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords

In [3]:
dataset = load_dataset("imsoumyaneel/sentiment-analysis-llama2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/89.0 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/173M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/598298 [00:00<?, ? examples/s]

In [4]:
# converting the training dataset to a dataframe
df = pd.DataFrame(dataset.get("train"))

In [6]:
max_length_manual = 0
i = 0
for idx, sentence in enumerate(df["sentence"]):
  if len(sentence) > max_length_manual:
    max_length_manual = len(sentence)
    i = idx

# getting the max length of a sentence
max_length = df["sentence"].apply(len).max()


In [7]:
# mapping the labels to numbers
df["mapped_label"] = df["label"].map({'joy': 0, 'neutral': 1, 'sadness': 2, 'anger': 3, 'fear': 4, 'love': 5, 'surprise': 6})
labels = df["mapped_label"].astype("float32").values

In [9]:
nltk.download('stopwords')
stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
def remove_stopwords(sentence):
  return re.sub(r'[^\w\s]','', sentence)

In [11]:
df["sentence"].map(remove_stopwords)

,sentence
0,Ill throw out the garbage
1,So Dick how about getting some coffee for ton...
2,Come on you can at least try a little beside...
3,What s wrong with that Cigarette is the thin...
4,Not for me Dick
...,...
598293,You got banned for participating in a brigade
598294,A joke is subjective pal second of all you nee...
598295,Well Im glad youre out of all that now How awf...
598296,Everyone likes NAME


In [12]:
# map or apply can be used here to apply the function to a series or a dataframe
def remove_stopwords(sentence):
  new_sentence = ""
  for word in sentence.split(" "):
    if word not in stopwords:
      new_sentence += " " + word
  # print(new_sentence)
  return new_sentence

df['sentence'] = df['sentence'].map(remove_stopwords)

In [13]:
# tokenize the words in sentences
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(df["sentence"])
tokenized_sentences = tokenizer.texts_to_sequences(df["sentence"])

In [14]:
print("0th text: ", tokenizer.sequences_to_texts(tokenized_sentences)[0])
print("0th token: ", tokenized_sentences[0])
print("actual 0th text: ", df['sentence'][0])

0th text:  i'll throw garbage
0th token:  [288, 1158, 3051]
actual 0th text:   I'll throw garbage .


In [15]:
# pad the smaller sentences
vocab_size = len(tokenizer.word_index)
padded_tokenized_sentences = tf.keras.preprocessing.sequence.pad_sequences(tokenized_sentences,
                                                                 maxlen=200,
                                                                 padding='post')

In [16]:
# modeling
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size,
                              output_dim=7,
                              input_length=200,
                              embeddings_initializer='uniform'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_tokenized_sentences, labels, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

print(labels.shape)

(418808, 200)
(179490, 200)
(418808,)
(179490,)
(598298,)


In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=7, validation_data=(X_test, y_test))

Epoch 1/10
 8533/59830 ━━━━━━━━━━━━━━━━━━━━ 59s 1ms/step - accuracy: 0.2371 - loss: -891.3461

KeyboardInterrupt: 